# Analysis Notebook for Ankan_Pal

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# --- Configuration ---
CSV_DIR = 'csv_files'
IMG_DIR = 'outputs'

# Ensure output directory exists
os.makedirs(IMG_DIR, exist_ok=True)

print("Loading Data...")

# --- 1. Load Data ---
try:
    trader_df = pd.read_csv(f"{CSV_DIR}/historical_data.csv")
    sentiment_df = pd.read_csv(f"{CSV_DIR}/fear_greed_index.csv")
    
    print("Data loaded successfully.")
    
    # --- 2. Preprocessing ---
    # Convert timestamps
    trader_df['dt'] = pd.to_datetime(trader_df['Timestamp IST'], format='%d-%m-%Y %H:%M', errors='coerce')
    trader_df['date_only'] = trader_df['dt'].dt.date

    sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])
    sentiment_df['date_only'] = sentiment_df['date'].dt.date

    # Merge Datasets
    merged_df = pd.merge(trader_df, sentiment_df, on='date_only', how='left')
    
    # Drop rows where we don't have sentiment data
    merged_df.dropna(subset=['classification'], inplace=True)
    
    print(f"Merged Data: {len(merged_df)} rows ready for analysis.")
    
    # --- 3. Visualization ---
    sns.set(style="whitegrid")
    sentiment_order = ['Extreme Fear', 'Fear', 'Neutral', 'Greed', 'Extreme Greed']
    existing_sentiments = [s for s in sentiment_order if s in merged_df['classification'].unique()]

    # Chart 1: PnL by Sentiment
    plt.figure(figsize=(10, 5))
    sns.barplot(x='classification', y='Closed PnL', data=merged_df, order=existing_sentiments,
                estimator=sum, errorbar=None, palette="RdYlGn", hue='classification', legend=False)
    plt.title('Total Realized PnL by Market Sentiment')
    plt.ylabel('Total Closed PnL (USD)')
    plt.tight_layout()
    plt.savefig(f"{IMG_DIR}/pnl_by_sentiment.png")
    plt.show()
    
    # Chart 2: Volume by Sentiment
    plt.figure(figsize=(10, 5))
    sns.countplot(x='classification', data=merged_df, order=existing_sentiments,
                  palette="viridis", hue='classification', legend=False)
    plt.title('Trading Volume (Number of Trades)')
    plt.ylabel('Count')
    plt.tight_layout()
    plt.savefig(f"{IMG_DIR}/activity_by_sentiment.png")
    plt.show()

except FileNotFoundError:
    print("Error: CSV files not found. Please ensure 'csv_files/' directory exists and contains data.")
except Exception as e:
    print(f"An error occurred: {e}")
